In [2]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [16]:
import pandas as pd
import logging
import general_functions.databricks_client as db_client
from general_functions.return_account_ids import return_account_ids
from general_functions.call_api_with_account_id import send_to_innkeepr_api_paginated

In [30]:
customer = "Vioneers"
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]
api_url = "https://targeting.innkeepr.ai/api"
min_date="2025-11-20"

In [31]:
response = send_to_innkeepr_api_paginated(
        f"{api_url}/profiles/query",
        account_id,
        {"lastModified": {"$gte": min_date}},
        logging.getLogger(),
    )

In [39]:
profiles = pd.json_normalize(response)

In [40]:
profiles

In [41]:
cookieid = "id"
col_name = "name"
col_cookie_date = "created"
data_col_extIds = "externalIds"

In [42]:
profiles = profiles.explode(data_col_extIds)

In [36]:
profiles[[cookieid, col_name, col_cookie_date]] = profiles[data_col_extIds].apply(
        lambda x: pd.Series([x[cookieid], x[col_name], x[col_cookie_date]]) if col_cookie_date in x.keys() else  pd.Series([x[cookieid], x[col_name]])
    )

In [37]:
profiles

In [43]:
profiles[cookieid] = [x.get(cookieid) for x in profiles[data_col_extIds]]
profiles[col_name] = [x.get(col_name) for x in profiles[data_col_extIds]]
profiles[col_cookie_date] = [x.get(col_cookie_date) for x in profiles[data_col_extIds]]

In [44]:
profiles

In [46]:
profiles = profiles[
        (profiles[col_name].str.contains("email_sha256"))
        | (profiles[col_name].str.contains("email"))
        | (profiles[col_name].str.contains("_ga"))
        | (profiles[col_name].str.contains("_fbp"))
        | (profiles[col_name].str.contains("criteo_gum_id"))
        | (profiles[col_name].str.contains("gclid"))
]
profiles